In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import json

### Cargar business.parquet

In [9]:
# Carga el archivo .parquet en un DataFrame de Pandas
df_business = pd.read_parquet('..//data//business.parquet')

### Preparar el achivo review.parquet

In [7]:
def convert_and_split_json_to_parquet(json_file_path, output_prefix, num_parts):
    # Contadores
    line_count = 0
    part_count = 0

    # Lee el archivo JSON en chunks
    with open(json_file_path, 'r', encoding='utf-8') as file:
        # Obtén el número total de líneas en el archivo
        total_lines = sum(1 for line in file)
        file.seek(0)

        # Calcula el tamaño del chunk
        chunk_size = total_lines // num_parts

        # Inicializa una lista para almacenar los registros
        records = []

        for line in file:
            # Parse la línea del JSON
            record = json.loads(line.strip())
            records.append(record)
            line_count += 1

            # Si hemos alcanzado el tamaño del chunk o es la última línea
            if line_count % chunk_size == 0 or line_count == total_lines:
                part_file_path = f"{output_prefix}_part{part_count + 1}.parquet"
                
                # Convertir la lista de registros a DataFrame
                df_chunk = pd.DataFrame(records)
                
                # Convertir el DataFrame a tabla Arrow y escribir a Parquet
                table = pa.Table.from_pandas(df_chunk)
                pq.write_table(table, part_file_path)
                print(f"Part {part_count + 1} written to {part_file_path}")
                
                # Resetear la lista de registros y aumentar el contador de partes
                records = []
                part_count += 1

In [8]:
# Usar la función
convert_and_split_json_to_parquet('..//data//review.json', 'review_output', 5)

Part 1 written to review_output_part1.parquet
Part 2 written to review_output_part2.parquet
Part 3 written to review_output_part3.parquet
Part 4 written to review_output_part4.parquet
Part 5 written to review_output_part5.parquet


In [10]:
# Lista de archivos .parquet
file_paths = [
    '..//code//review_output_part1.parquet',
    '..//code//review_output_part2.parquet',
    '..//code//review_output_part3.parquet',
    '..//code//review_output_part4.parquet',
    '..//code//review_output_part5.parquet'
]

In [11]:
# Filtrar cada DataFrame y almacenarlos en una lista
filtered_dfs = []
for file_path in file_paths:
    # Leer el archivo .parquet
    df_part = pd.read_parquet(file_path)
    
    # Filtrar el DataFrame
    df_part_filtered = df_part[df_part['business_id'].isin(df_business['business_id'])]
    
    # Agregar el DataFrame filtrado a la lista
    filtered_dfs.append(df_part_filtered)
    
    # Guardar el DataFrame filtrado en un archivo .parquet
    filtered_file_path = file_path.replace('.parquet', '_filtered.parquet')
    df_part_filtered.to_parquet(filtered_file_path)
    print(f"Filtered file written to {filtered_file_path}")

Filtered file written to ..//code//review_output_part1_filtered.parquet
Filtered file written to ..//code//review_output_part2_filtered.parquet
Filtered file written to ..//code//review_output_part3_filtered.parquet
Filtered file written to ..//code//review_output_part4_filtered.parquet
Filtered file written to ..//code//review_output_part5_filtered.parquet


In [12]:
# Combinar los DataFrames filtrados
df_review = pd.concat(filtered_dfs, ignore_index=True)

In [13]:
# Guardar el DataFrame combinado en un archivo .parquet
combined_file_path = '..//data//review.parquet'
df_review.to_parquet(combined_file_path)

### Ya filtrado y guardado como .parquet y para los Starbuks de todo el pais

In [15]:
# Carga el archivo .parquet en un DataFrame de Pandas
df_review = pd.read_parquet('..//data//review.parquet')

In [16]:
# Función para normalizar y capitalizar texto
def normalize_and_capitalize(text):
    if text is None:
        return ''
    # Convertir a minúsculas y luego capitalizar la primera letra de cada palabra
    normalized_text = ' '.join([word.capitalize() for word in text.lower().split()])
    return normalized_text

In [17]:
# Asegurarse de que la columna 'text' está convertida a str y luego aplicar la función
df_review['text'] = df_review['text'].astype(str).apply(normalize_and_capitalize)

In [18]:
# Verificar los primeros registros de la columna 'text' después de la transformación
print(df_review['text'].head())

0    Ordered Caramel Frappe At Drive Thru, Big Mist...
1    Drum-roll Please! Review #100 Coming Right Up!...
2    We Stopped Here For My Chai And Hubby's Coffee...
3    There's Been Three Times That I've Ordered A G...
4    I Went In When They Had 4 People Working, Wait...
Name: text, dtype: object


In [19]:
# Convertir la columna 'date' a tipo datetime
df_review['date'] = pd.to_datetime(df_review['date'], errors='coerce')

In [20]:
# Verificar los primeros registros de la columna 'date' después de la conversión
print(df_review['date'].head())

0   2017-12-29 19:38:31
1   2014-02-05 19:38:24
2   2017-02-09 04:35:39
3   2016-08-25 14:08:18
4   2016-01-30 01:10:42
Name: date, dtype: datetime64[ns]


In [21]:
# Restablecer el índice
df_review = df_review.reset_index(drop=True)

In [22]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21739 entries, 0 to 21738
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   review_id    21739 non-null  object        
 1   user_id      21739 non-null  object        
 2   business_id  21739 non-null  object        
 3   stars        21739 non-null  float64       
 4   useful       21739 non-null  int64         
 5   funny        21739 non-null  int64         
 6   cool         21739 non-null  int64         
 7   text         21739 non-null  object        
 8   date         21739 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 1.5+ MB


In [23]:
df_review.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,ULzbgT5SPThiMOoo4a_Adw,veBX5roHQLNnQ1rTtNh_gg,aJvxWyQIG5OLfBw3qAe8xA,2.0,0,0,0,"Ordered Caramel Frappe At Drive Thru, Big Mist...",2017-12-29 19:38:31


In [24]:
# Guardar el DataFrame como archivo Parquet
df_review.to_parquet('..//data//review.parquet')